In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Import from the crewAI libray.
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# import library
from dotenv import load_dotenv
import os #needed to call function
import openai

In [7]:
# setup configuration function
def configure():
    load_dotenv()
    
# load configuration
configure()

# get the key
openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'


## Creating Agents

In [8]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [11]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [12]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [16]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [17]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [18]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [22]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2025-08-31 06:54:47,984 - 8535219968 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [24]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'PCTY',
    'initial_capital': '5000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [25]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (PCTY). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze market data for PCTY and identify trends, I need to get a comprehensive data analysis done which includes statistical modeling and machine learning. I will delegate this task to the Data Analyst.

Action: Delegate work to co-worker
Action Input: {
  "coworker": "Data Analyst",
  "task": "Continuously monitor and analyze market data for the stock PCTY using statistical modeling and machine learning to identify trends and predict market movements",
  "context": "The task involves gathering historical and real-time data on PCTY, employing statistical models and machine learning algorithms to analyze this data, and identifying any significant trends or patterns that could in

In [26]:
from IPython.display import Markdown
Markdown(result)

The comprehensive risk analysis report for the trading strategies concerning PCTY, under current bearish market conditions, details the following risks and mitigation strategies:

**Put Options Strategy Risks and Mitigation:**
1. **Risk of Overpriced Options Due to Increased Implied Volatility (IV):** As the earnings release date (October 29, 2025) approaches, there's a significant risk of increased IV which may result in overpriced options. This discrepancy, especially when compared to Historical Volatility (HV), indicates a potential overvaluation of put options.
   - **Mitigation Strategy:**
     - **Close Monitoring of IV and HV:** Traders should closely monitor these volatility metrics in the weeks leading to the earnings release. Comparing these metrics to historical averages will aid in determining if the options are overpriced.
     - **Option Selling Strategy:** Consider selling options when IV is exceptionally high, capitalizing on the expected decline in volatility post-earnings, thus potentially maximizing returns from volatility contractions.

**Defensive Trading Strategy Risks and Mitigation:**
1. **Risk of Slower Growth in Resilient Sectors:** Investing in sectors like utilities, healthcare, and consumer staples may lead to slower growth rates, especially in a bearish market where these sectors traditionally underperform compared to more volatile sectors.
   - **Mitigation Strategy:**
     - **Dynamic Stop-Loss Orders:** Implement stop-loss orders that are responsive to real-time market conditions. This will protect the investment from significant losses during sudden market downturns.
     - **Diversification Within Sectors:** Spread investments across various companies and sub-sectors within the resilient sectors to minimize risks associated with any single underperforming area.
     - **Ongoing Portfolio Review and Adjustment:** Regularly reassess and adjust the portfolio based on continuous risk evaluation and market analyses. This proactive approach will help in responding effectively to changes in market sentiment and economic indicators.

By thoroughly addressing these risks and implementing the suggested mitigation strategies, the firm is better positioned to align its trading activities with its medium risk tolerance and the overarching goal of capital preservation, even in a bearish market environment for PCTY.